In [2]:
from sklearn.ensemble import RandomForestClassifier as rnf
import numpy as np, matplotlib.pyplot as plt
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
import pandas as pd
idx = 50
scale = 1
x1 = np.random.normal([-2, 1], scale = scale, size = (idx,2))
x2 = np.random.normal([2, 1], scale = scale, size = (idx, 2))
x  = np.vstack((x1, x2))
y = np.zeros(len(x))
cmap = {0: 'red', 1 : 'blue'}
y[:idx] = 1
X = np.hstack((x,y[:, None]))
df = pd.DataFrame(X, columns = 'x1 x2 y'.split())

df['colors'] = df['y'].map(cmap)
r = np.linspace(-3, 3)
from bokeh.plotting import figure, save
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import CustomJS, Slider
from bokeh.layouts import row, gridplot, column


output_notebook()
idx = 400
start = -3
fig = figure(toolbar_location = 'below', plot_height = idx, plot_width = idx, \
             x_range = (x[:, 0].min(), x[:, 0].max()))
vl1 = fig.circle(np.ones(r.shape) * start,\
                 np.linspace(x[:, 1].min(), x[:, 1].max()\
                ))

fig.circle(x = 'x1', y = 'x2', source = df, fill_color = 'colors', line_color = None, size = 20)


hists = [figure(plot_width = fig.plot_width // 2, plot_height = fig.plot_height // 2 , y_range = (0, 1.1), \
                title = name) for name in [r'left', r'right']]


entr = figure(plot_height = fig.plot_height // 2, plot_width = fig.plot_width, \
              y_range = (0, 0.5), x_range = (x[:, 0].min(), x[:, 0].max()),\
              output_backend = 'webgl')
entr.yaxis.axis_label = "Entropy"

vl2  = entr.circle(np.ones(r.shape) * start,\
                 np.linspace(0, 1, r.size),\
               )

vl3 = entr.scatter([], [], size = 20, \
               )
def getDist(threshold):
    # subset data
    hs = []
    for f in [lambda x: x < threshold, lambda x: x >= threshold]:
        idx = np.where(f(x[:, 0]))
        t   = y [idx]
        counts = tuple((t ==i).sum() for i in [0, 1])
        hs.append((*np.histogram(t, bins = [-0.5, 0.5, 1.5], density = 1), np.array(counts)\
                  ))
    return hs
            
        
for idx, (height, edges, _) in enumerate(getDist(start)):
    print(_)
    hists[idx].quad(top = height, left = edges[:-1], \
            right = edges[1:], bottom = 0, name = f'hist{idx}',\
           line_color = 'black', fill_color = [*cmap.values()])
    
computed = {}
def update_plot(x):
    y = vl1.data_source.data['y']
    vl1.data_source.data['x'] = [x for i in y]
    
    
    vl2.data_source.data['x'] = [x for i in y]
    H = 0
    for idx, (h, edges, counts) in enumerate(getDist(x)):
        tmp = hists[idx].select_one({'name' : f'hist{idx}'})
        H   += - np.nansum((np.log(h) * h * counts / 100))
        tmp.data_source.data['top'] = h
    
    if x not in computed:
        vl3.data_source.data['x'] += [x]
        vl3.data_source.data['y'] += [H]
#         entr.scatter(x, H, color = 'black', size = 20)
        computed[x] = True
   
        
    push_notebook(handle = bokeh_handle)  
    
    
callback = CustomJS(code="""
if (IPython.notebook.kernel !== undefined) {
    var kernel = IPython.notebook.kernel;
    cmd = "update_plot(" + cb_obj.value + ")";
    kernel.execute(cmd, {}, {});
}
""")

slider = Slider(start = r.min(), 
                end = r.max(),
                value = start,
                step = .25,
                callback = callback, width = fig.plot_width,)

model = gridplot([[column(fig)], [row(*hists)]])

model = row([column(fig, entr, slider), column(*hists), ])
# model = row(hists[0], column(slider, fig), hists[1],)
bokeh_handle = show(model, notebook_handle=True)


Loading BokehJS ...

[0 8]
[50 42]


In [119]:
computed

{-2.5: True,
 -2: True,
 -1.5: True,
 -1: True,
 -0.49999999999999956: True,
 0: True,
 0.5: True,
 1: True,
 1.5: True,
 2.000000000000001: True,
 -3: True,
 2.5: True,
 3: True}

In [117]:
vl3.update()